In [4]:
#import libraries
import requests
import pandas as pd
import time
import numpy as np
import re

In [5]:
#Keys
API_KEY = '3b717b60d6778662c2b416c47c8f0067'

bills = pd.read_excel('/kaggle/input/policing-legis-bill/Policing Legislation.xlsx')

getSearch = '&op=getSearch&state='
getYear = '&year='
url = 'https://api.legiscan.com/?key='+API_KEY
# https://api.legiscan.com/?key=3b717b60d6778662c2b416c47c8f0067&op=getSearch&state=CA&bill=SB7
# https://api.legiscan.com/?key=3b717b60d6778662c2b416c47c8f0067&op=getSearch&state=CA&year=2019&bill=SB7

In [6]:
def map_status(df):
  status_mapping = {
      0: 'Pre-filed or pre-introduction',
      1: 'Introduced',
      2: 'Engrossed',
      3: 'Enrolled',
      4: 'Passed',
      5: 'Vetoed',
      6: 'Failed',
      7: 'Override',
      8: 'Chaptered',
      9: 'Refer',
      10: 'Report Pass',
      11: 'Report DNP',
      12: 'Draft',
  }

  # Map status values to descriptions and add to the Status_Label column
  df['Status_Label'] = df['Status_Code'].map(status_mapping)
  return get_status(df)

In [7]:
class BillInfo:
    def __init__(self, year, state, bill_number):
        self.year = year
        self.state = state
        self.bill_number = bill_number
        self.bill_head = f"{self.year} {self.state} {self.bill_number}"

In [8]:
def get_status(data):
  conditions = [
      (data['Status_Code'] == 4),
      (data['Status_Code'] == 6)
  ]
  values = ['Enacted', 'Failed']

  # Apply conditions using np.select()
  df['Status'] = np.select(conditions, values, default='Pending')
  return df

In [9]:
def get_bill_details(billId):
  getBill = '&op=getBill&id='+ str(billId)
    #testing GetBill API call
  response_bill = requests.get(url+getBill).json()
  bill_St_code = response_bill.get('bill', {}).get('status')
  bill_St_date = response_bill.get('bill', {}).get('status_date', 'N/A')
  time.sleep(0.1)

  return bill_St_code, bill_St_date

In [10]:
# def get_bill(df2, getSearch, State, year, bill_num):
#   url = 'https://api.legiscan.com/?key='+API_KEY
#   response = requests.get(url+getSearch+State+year+bill_num).json( )
#   time.sleep(1)
#     #Inicialize Dictionary
#   search_results = response.get('searchresult', {})

#     # Loop through each search result
#   for index, result in search_results.items():
#     if index != '0':
#       billNo = result.get('bill_number')
#       billId = result.get('bill_id')
#       billTitle = result.get('title')
#       billDate = result.get('last_action_date')

#       bill_St_code ,bill_St_date = get_bill_details(billId)

#         #save to pandas df
#       df2 = df2.append({'Bill_Number':billNo,'Bill_Title':billTitle, 'Bill_Id':billId,
#                         'Status_Code':bill_St_code,'Status_Date':bill_St_date,
#                         'Bill_Date':billDate},
#                       ignore_index = True)
#   return df2

def get_bill(df2, getSearch, State, year, bill_num):
  url = 'https://api.legiscan.com/?key='+API_KEY
  response = requests.get(url+getSearch+State+"&year="+year+"&bill="+bill_num).json( )
  try:
    billNo = response['searchresult']['0']['bill_number']
    billId = response['searchresult']['0']['bill_id']
    billTitle = response['searchresult']['0']['title']
    billDate = response['searchresult']['0']['last_action_date']
    bill_St_code ,bill_St_date = get_bill_details(billId)
  except KeyError:
    print("Bill does not contain expected data. State: " + State + " year: " + year + " Bill: " + bill_num)
    return df2

        #save to pandas df
  df2 = df2.append({'Bill_Number':billNo,'Bill_Title':billTitle, 'Bill_Id':billId,
                        'Status_Code':bill_St_code,'Status_Date':bill_St_date,
                        'Bill_Date':billDate},
                      ignore_index = True)
  return df2

In [11]:
# def split_column_values(data):
#     year_state_bill = []

#     for value in data:
#         # Use regular expressions to extract the year, state abbreviation, and bill number
#         match = re.match(r'(\d{4})\s+([A-Z]{2})\s+(.*)', value)
#         if match:
#             year = match.group(1)
#             state = match.group(2)
#             bill = match.group(3)
#             year_state_bill.append((year, state, bill))
#         else:
#             year_state_bill.append(("N/A", "N/A", "N/A"))  # Handle cases where the pattern doesn't match

#     return year_state_bill
def split_column_values(data):
    bill_info_list = []

    for value in data:
        # Use regular expressions to extract the year, state abbreviation, and bill number
        # (\d{4})\s+([A-Z]{2})\s+(.*)
        match = re.match(r'(\d{4})\s+([A-Z]{2})\s+(.*)', str(value))
        if match:
            year = match.group(1)
            state = match.group(2)
            bill = re.sub(r'\s|\.','', match.group(3))
            #bill = re.sub(r'\s|\.','', match.group(3))

            bill_match = re.match(r'([A-Za-z]+\d+)', bill)
            if bill_match:
              bill = bill_match.group(1)

            bill_info = BillInfo(year, state, bill)
            bill_info_list.append(bill_info)
        else:
            bill_info_list.append(BillInfo("N/A", "N/A", "N/A"))  # Handle cases where the pattern doesn't match

    return bill_info_list

In [29]:
##%%capture
#supossed to be main
#build the dataframe
# https://api.legiscan.com/?key=3b717b60d6778662c2b416c47c8f0067&op=getSearch&state=CA&year=2019&bill=SB7
# 

# us_state_abbreviations = [
#     'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
#     'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
#     'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
#     'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
#     'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
# ]

df = pd.DataFrame(columns =["Bill_Number", "Bill_Title","Bill_Id","Bill_Date","Status","Status_Code","Status_Label", "Status_Date"])
missing_df = pd.DataFrame(columns = ["Missing Bills", "Bill_State", "Bill_Year","Bill_Number"])
input_data = bills['Bill'].tolist()

#try 2700 in index if it works
bill_info_list = split_column_values(input_data[:3822])
count = 0
count_missing = 0

for bill_info in bill_info_list:
    print("Called bill: " + bill_info.state, bill_info.year, bill_info.bill_number)
    count = count + 1
    if bill_info.bill_number != "N/A":
      df = get_bill(df, getSearch, bill_info.state, f'{bill_info.year}', bill_info.bill_number)
#     if bill_info.bill_number == "N/A":
#         missing_df = missing_df.append(bill_info.bill_head, bill_info.state, bill_info.year, bill_info.bill_number)
    if bill_info.bill_number == "N/A":
        new_entry = {'Missing Bills': bill_info.bill_head, 'Bill_State': bill_info.state, 'Bill_Year': bill_info.year, 'Bill_Number': bill_info.bill_number}
        missing_df = missing_df.append(new_entry, ignore_index=True)
#         count_missing = count_missing + 1
    
# print("Number of Bills Missing: ",count_missing)
# print("Number of Bills : ",count)

df['Bill'] = bills['Bill']

Called bill: AL 2018 HB105


/tmp/ipykernel_32/835514552.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'Bill_Number':billNo,'Bill_Title':billTitle, 'Bill_Id':billId,


Called bill: AL 2018 HB149


/tmp/ipykernel_32/835514552.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'Bill_Number':billNo,'Bill_Title':billTitle, 'Bill_Id':billId,


Called bill: AL 2018 HB33


...

/tmp/ipykernel_32/835514552.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'Bill_Number':billNo,'Bill_Title':billTitle, 'Bill_Id':billId,


Called bill: NY 2021 AB5621


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Called bill: WI 2021 SJR79


/tmp/ipykernel_32/835514552.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'Bill_Number':billNo,'Bill_Title':billTitle, 'Bill_Id':billId,


In [31]:
df = map_status(df)
df

# df.to_csv (r'Update_status.csv', index=None)
#2630(3000) out of 3822 rows df

,Bill_Number,Bill_Title,Bill_Id,Bill_Date,Status,Status_Code,Status_Label,Status_Date,Bill
0,HB105,"Emergency responders, certified peer support m...",1049951,2018-03-22,Enacted,4,Passed,2018-03-22,2018 AL H.B. 105
1,HB149,"Poarch Creek Indians, granted police powers un...",1049942,2018-03-21,Enacted,4,Passed,2018-03-21,2018 AL H.B. 149
2,HB33,"Calhoun Co., municipalities not located entire...",1033179,2018-01-30,Enacted,4,Passed,2018-01-30,2018 AL H.B. 33
3,HB449,"Public education, volunteer emergency security...",1095582,2018-02-22,Pending,1,Introduced,2018-02-22,2018 AL H.B. 449
4,HJR101,"McGhee, Greg, Scottsboro Police Department Lif...",1069281,2018-02-01,Enacted,4,Passed,2018-02-01,2018 AL H.J.R 101
...,...,...,...,...,...,...,...,...,...
3292,AB746,Public access to records concerning the conduc...,1537378,2022-03-15,Failed,6,Failed,2022-03-15,2019 RI H.B. 7049
3293,AB777,Grants to political subdivisions for additiona...,1537901,2022-05-17,Failed,6,Failed,2022-05-17,2019 RI H.B. 7089
3294,SB120,Law enforcement agency policies on the use of ...,1456074,2021-08-09,Enacted,4,Passed,2021-08-09,2019 RI H.B. 7139
3295,SB471,Crisis training for law enforcement officers. ...,1520952,2022-03-15,Failed,6,Failed,2022-03-15,2019 RI H.B. 7203


In [ ]:
df = map_status(df)
df
# df.to_csv (r'Update_status.csv', index=None)

In [ ]:
#Check which states are not working on this format
    #do I do this in the API resquest or after getting all the info?
    #AZ, AR, CA, CT, DC*, FD*, FL, GA, HI*, IA, KY*, ME*, MA*, MN, MS, MO, MT*, NV*, NH*
    #States with *
#Check cases where states are working but some certain bills are not
#     47 bill_info.bill_number == "N/A" cases
#     3297 Bills in current code
#     478 Bills with non-working API Calls
#Check boss code (even though I can't understand shit)